In [ ]:
import pandas as pd
import numpy as np
import os

import config as cfg
from utils.yahoo_downloader import YahooDownloader
import utils.helper_functions as hf

""" from stockstats import wrap

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import mutual_info_classif """

pd.options.mode.chained_assignment = None

In [ ]:
folder_path = './db/'
file_name = 'ohlcv.pkl'
file_path = folder_path + file_name

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

if os.path.isfile(file_path):
    df = pd.read_pickle(file_path)

else: # 2 minutes
    df = YahooDownloader(cfg.start_date, cfg.end_date, cfg.tickers).fetch_data()        
    df.to_pickle(file_path)
    df.to_csv(folder_path + 'ohlcv.csv')

df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.strftime('%Y')
print(df)

In [ ]:
print(df.dtypes)

In [ ]:
def add_supports_resistances(df: pd.DataFrame) -> pd.DataFrame:
    df['rolling_min'] = hf.get_rolling_min(df['low'], cfg.target_days)
    df['rolling_max'] = hf.get_rolling_max(df['high'], cfg.target_days)
    df['last_close'] = df['close'].shift(1)

    df['pivot'] = hf.get_pivot(df['rolling_max'], df['rolling_min'], df['last_close'])

    df = df.dropna()

    df['support1'] = hf.get_support1(df['pivot'], df['rolling_max'])
    df['support2'] = hf.get_support2(df['pivot'], df['rolling_max'], df['rolling_min'])
    df['resistance1'] = hf.get_resistance1(df['pivot'], df['rolling_min'])
    df['resistance2'] = hf.get_resistance2(df['pivot'], df['rolling_max'], df['rolling_min'])

    return df

In [ ]:
support_column = 'support2'
resistance_column = 'resistance1'

def init_support_resistance(df: pd.DataFrame) -> list:
    support = df.iloc[0][support_column]
    resistance = df.iloc[0][resistance_column]
    return support, resistance

def update_support_resistance(row_df: pd.DataFrame) -> list:
    support = row_df.__getattribute__(support_column)
    resistance = row_df.__getattribute__(resistance_column)
    return support, resistance

def get_profit(df: pd.DataFrame) -> list:
    profit = 1
    buy_price = 0
    bought_days = 0
    is_bought = False

    support, resistance = init_support_resistance(df)

    for row in df.itertuples():
        last_close = row.last_close
        open = row.open

        if (is_bought and last_close > resistance):
            is_bought = False
            sell_price = open
            profit *= (sell_price / buy_price) * cfg.fee_coef
        elif (not is_bought and last_close < support):
            is_bought = True
            buy_price = open
            profit *= cfg.fee_coef
        
        if (last_close > resistance or last_close < support):
            support, resistance = update_support_resistance(row)

        if is_bought:
            bought_days += 1

    return profit - 1, bought_days

In [ ]:
profits = []
available_tickers = df['tic'].unique()

for ticker in available_tickers:
    ticker_df = df.loc[df['tic'] == ticker]
    ticker_df = add_supports_resistances(ticker_df)

    years = ticker_df['year'].unique()

    for year in years:
        year_df = ticker_df.loc[ticker_df['year'] == year]

        days = len(year_df)

        if days > 0:
            profit, bought_days = get_profit(year_df)

            profits.append({
                'ticker_year': f'{ticker}-{year}',
                'profit': hf.pct(profit),
                'daily_profit': hf.pct(profit / bought_days) if bought_days > 0 else 0,
                'days': days,
                'bought_days': bought_days,
                'bought_days%': hf.pct(bought_days / days)
            })

profit_df = pd.DataFrame(profits)
print(profit_df.head())
print(f"Average profit: {profit_df['profit'].mean(skipna=True)}%")
print(f"Average daily profit %: {profit_df['daily_profit'].mean(skipna=True)}%")
print(f"Average bought days %: {profit_df['bought_days%'].mean(skipna=True)} days")

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(profit_df)

In [ ]:
cac_profit_df = profit_df[profit_df['ticker_year'].str.startswith('^FCHI')]
print(cac_profit_df)
print(f"Average profit: {cac_profit_df['profit'].mean(skipna=True)}%")
print(f"Average daily profit %: {cac_profit_df['daily_profit'].mean(skipna=True)}%")
print(f"Average bought days %: {cac_profit_df['bought_days%'].mean(skipna=True)} days")